# Fake news detection

In [2]:
# Dependancies
import pandas as pd
import numpy as np

import re
import nltk

nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


stpwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to /home/ds/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data understanding

The dataset contains 20800 rows x 5 columns:
- id: each entry contains and ID
- title: news title
- author: author's name/last name
- text: news content
- label: this is our target column, 1 = unreliable news & 0 = reliable news

In [5]:
# Loading CSV into pandas
df = pd.read_csv("NB-fakenews.csv")


In [6]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [7]:
# Checking missing values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

# Data preparation 

title & author are missing some values, however it is a small amount in comparison to 20800 rows. 

The missing entries will be replaced with NULL values. To be able to predict the label column the title & author columns will be used. However both columns are currently containing text values with many characters and symbols, which is making not usable for our model.

Steps:

- replace missing values with NULL
- merge title and author into a new column



In [9]:
#  Replacing missing values with isnull

df = df.fillna("")
df.isnull().sum()



id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
# Merging author and title column

df['merged'] = df['author']+' '+ df['title']
df

,id,title,author,text,label,merged
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [18]:
# Checking the value caount of label
df['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [23]:
# newly merged column: title and author
df['merged']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: merged, Length: 20800, dtype: object

In [24]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
merged    0
dtype: int64

In [26]:
# The target lebels are in the label column 0= reliable 1= unrelaible

x = df.drop(columns = 'label', axis = 1)
y = df['label']

In [27]:
x.head(2)

,id,title,author,text,merged
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."


In [28]:
y.head(2)

0    1
1    0
Name: label, dtype: int64

The dataframe is set x variable excuding labels
The label is set to y from dataframe

## Stemming: process of reducing a word to its root word

Example: Action, Actor, Acting = Act

From the merged column we will extract/remove all characters/symbols that are not in a to z or A to Z. This is needed because we will use the the words in merged columns to vectorize our data in merged, so that we can apply our prediction model. 

Steps:

- Define a function that 1) subtracts symbols and unneeded characters, 2) makes words in lower case, splits the words and sets into a list. 3) Loops through the list to remove meaningless words (that are in the list of stopwords module)
- Create the model

In [31]:
stem = PorterStemmer()

In [36]:
def stemming(merged):
    stemm_merged = re.sub('[^a-zA-Z]', ' ', merged) # Substitude eveyry character excluding a-zA-Z
    stemm_merged = stemm_merged.lower() # make lower case 
    stemm_merged = stemm_merged.split() # split and make list
    stemm_merged = [stem.stem(word) for word in stemm_merged if not word in stopwords.words('english')] # make a new list of words that are not stopwords (meaningless)
    stemm_merged = ' '.join(stemm_merged)
    return stemm_merged
    

In [48]:
df['merged']

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: merged, Length: 20800, dtype: object

In [46]:
df['merged'] = df['merged'].apply(stemming)

In [47]:
df['merged']

0        darrel lucu hou dem aid even see comey letter ...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exerci b...
20799                            david swanson keep f aliv
Name: merged, Length: 20800, dtype: object

The merged column is ready to be transformed into vectors. Since the model will make the predictions based on only author & title columns which is the merged column now, we will create a new dataframe for merged and another one for the labels. 

In [49]:
x = df['merged'].values
y = df['label'].values

In [56]:
print(x, len(x))

['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv'] 20800


In [58]:
print(y, len(y))

[1 0 1 ... 0 1 1] 20800


In [59]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x = vectorizer.transform(x)

In [63]:
x.shape

(20800, 16973)

# Creating the model

Let start with splitting the data into train and test:

In [66]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 2)

Now we have 4 datasets: 
- x_train (80% of df['merged']), 
- x_test(20% of df['merged']),
- y_train (80% of df['lebel']),
- y_test (20% of df['lebel'])

The model is using Logistic Regression Classifier to classify merged column into 0 or 1, reliable or unreliable. 

In [74]:
model = LogisticRegression()

In [75]:
model.fit(x_train, y_train) 

LogisticRegression()

# Deployment and Evaluation

In [83]:
# Accuracy on training data
x_train_pred = model.predict(x_train)
training_accuracy = accuracy_score(x_train_pred, y_train)

In [84]:
print('The accuracy of the model on the training data is: ', training_accuracy)

The accuracy of the model on the training data is:  0.9866586538461538


Very high rate of accuracy on the training data, which makes sense because the model is trained on this dataset. Accuracy on the test data will tell how good the model is predicting. Lets measure the accuracy of the model on the test data:

In [85]:
# Accuracy on test data
x_test_pred = model.predict(x_test)
test_accuracy = accuracy_score(x_test_pred, y_test)

In [86]:
print('The accuracy of the model on the test data is: ', test_accuracy)

The accuracy of the model on the test data is:  0.9790865384615385


The models performs well with 98% of accuracy, it less then 1% inaccurate compare to the the score on the training data.